# Collaborative Filtering

Collaborative Filtering means that interests of other users are taking into account for making a playlist prediction. Spotify uses a matrix where each row is a user and each column is a song, each entry therefore resembles how often a user has listened to a song. Since this data is not publicly available, we will try the same approach, but with a different matrix. The matrix will have one row for each playlist and each column will be a song. The entries of the matrix are going to be 1 if the song is in the playlist and 0 if not. Read more about collaborative filtering [here](TODO).

Let's first import all important libraries. pandas DataFrames are used to store the data and numpy arrays for matrix computations.

In [13]:
import warnings
import numpy as np
import pandas as pd

from scripts.matrix_factorization import MF  # for matrix factorization

Load the raw data.

In [14]:
df = pd.read_csv("data/processed_data_train.csv")
print(f"We have a dataset of {len(df)} entries")

We have a dataset of 60732 entries


We want to build a matrix where each row is a playlist, and each column resembles a song. The dimensions of our matrix resemble those of the playlist number and track number.

In [15]:
num_playlists = df["name"].nunique()  # count distinct values, this is the number of playlists
num_tracks = df["track_name"].nunique()  # count distinct values, this is the number of tracks
print(f"Playlists: {num_playlists} \nTracks: {num_tracks}")

Playlists: 779 
Tracks: 27902


Group the data by the playlist name. This results in a Series.

In [16]:
playlists = df.groupby('name')["track_name"].apply(list)
playlists.head()

name
 CHiLl         [Fresh Eyes, i hate u, i love u (feat. olivia ...
 Frozen        [Queen Elsa of Arendelle - Score Demo, Reindee...
 indie rock    [Back In Your Head, Be Good (RAC Remix), Bambi...
#Relaxed       [Bag Lady, On & On, I Can't Stop Loving You, L...
#Workout       [Can't Feel My Face - Martin Garrix Remix, Ign...
Name: track_name, dtype: object

We need a list of the unique songs to create each playlist vector.

In [17]:
unique_songs = list(df["track_name"].unique())  # list of unique songs, maps each song to an index
print(f"Number of unique songs: {len(unique_songs)}")

Number of unique songs: 27902


We can now iteratively build our matrix by creating each playlist in a vector of its songs. This is done by one-hot encoding, meaning every column is a song, and every row a playlist, and the row-column combination is one if the song was added to the playlist. 

In [18]:
one_hot_playlists = list()
for playlist in playlists:
    playlist_array = np.zeros(num_tracks)
    for song in playlist:
        playlist_array[unique_songs.index(song)] = 1  # set array to 1 at index of the song
    one_hot_playlists.append(playlist_array)
one_hot_playlists = np.array(one_hot_playlists)  # convert to one numpy array (matrix)

For example, the first playlist "CHiLl" includes the song "Make Me (Cry)" and does not include "Mr. Brightside". Check if the value is one and zero respectively at the corresponding positions.

In [19]:
print(one_hot_playlists[0][unique_songs.index("Make Me (Cry)")] == 1.0)
print(one_hot_playlists[0][unique_songs.index("Mr. Brightside")] == 0.0)

True
True


The shape of our playlists should be playlist number times distinct track number.

In [20]:
one_hot_playlists.shape

(779, 27902)

We now apply matrix factorization on our data. This means, we try to find two matrices, which multiplied are as close to the original matrix as possible. We train using gradient descent, meaning we try to minimize the error in each iteration.

In [21]:
mf = MF(one_hot_playlists, K=2, alpha=0.1, beta=0.01, iterations=100)
mf.train()

Iteration: 10 ; error = 2.0281
Iteration: 20 ; error = 1.1173
Iteration: 30 ; error = 0.7482
Iteration: 40 ; error = 0.5483
Iteration: 50 ; error = 0.4339
Iteration: 60 ; error = 0.3413
Iteration: 70 ; error = 0.2817
Iteration: 80 ; error = 0.2381
Iteration: 90 ; error = 0.2051
Iteration: 100 ; error = 0.1777


[(0, 14.950884613801279),
 (1, 8.106850615677738),
 (2, 5.6156187089921605),
 (3, 4.34589675812203),
 (4, 3.6369515615855916),
 (5, 3.0704483719141593),
 (6, 2.721259858534696),
 (7, 2.426036324221732),
 (8, 2.21713954013998),
 (9, 2.028119560985024),
 (10, 1.8767216549766619),
 (11, 1.7429925483038409),
 (12, 1.6443373719828338),
 (13, 1.5337784269549746),
 (14, 1.4577419774451954),
 (15, 1.3697593945867361),
 (16, 1.302472751841376),
 (17, 1.2284534027093708),
 (18, 1.167544730299626),
 (19, 1.117332974074874),
 (20, 1.0781493639897863),
 (21, 1.0201660153555974),
 (22, 0.9864622149518064),
 (23, 0.9393765836974682),
 (24, 0.9017368970492708),
 (25, 0.8657816130041736),
 (26, 0.8313908622980881),
 (27, 0.812399769339396),
 (28, 0.773426103795423),
 (29, 0.7481871056269123),
 (30, 0.7210426180393957),
 (31, 0.6962544019272964),
 (32, 0.6749146914219895),
 (33, 0.659488473004516),
 (34, 0.634691653091266),
 (35, 0.6152434789410065),
 (36, 0.595898701434051),
 (37, 0.5801295108936905),


As a result, we get a matrix where values should be close to their original values, but unknown values, in our case songs that are not in the playlist, are approximated by the matrix factorization. For example, if we look at the same song from earlier, the value is close to 1. Looking in at a song that has not been added to the playlist, the value is now approximated to how likely it should be added to the playlist.

In [22]:
print(mf.full_matrix()[0][unique_songs.index("Make Me (Cry)")])
print(mf.full_matrix()[0][unique_songs.index("Mr. Brightside")])

0.9998369191681457
1.0001460949905643


We can store our Matrix and use it in the [evaluation notebook](evaluation.ipynb) for making recommendations.

In [23]:
with open('data/matrix.npy', 'wb') as f:
    np.save(f, mf.full_matrix())

Save track IDs and playlist names in order to be able to reconstruct the information.

In [24]:
track_name_to_uri = dict(zip(df.track_name, df.track_uri))
unique_songs_uris = [track_name_to_uri[x] for x in unique_songs]

with open('data/unique_songs_uris.txt', 'w') as f:
	f.write('\n'.join(unique_songs_uris))
    
with open('data/playlists.txt', 'w', encoding="utf-8") as f:
	f.write('\n'.join(list(playlists.index)))